In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17293,2024-04-06,Brasil Nbb,14:00,Fortaleza B.C.,R10 Score Vasco da Gama,1.81,1.91,153.5,1.85,1.85,-1.5,1.94,2.08,xGMaPyWa,0.552486,0.523560,0.540541,0.540541,0.076046,0.6,0.6,NaN,NaN,142.282,59.365018,0.417235,130.090,40.648609,0.312465,125.418,201.210,102.48,200.07,0.0,0.0,0.0,0.0,0.038016,0.000000,0.049251,-0.56,-0.112,-7.232143,0.687818,0.5,-0.187818,1.12,0.224,4.062500,0.000000,0.0,0.000000
17294,2024-04-06,Espanha Acb,12:00,Bilbao,Gran Canaria,2.11,1.73,162.5,1.80,1.94,1.5,1.93,1.58,MF653bfa,0.473934,0.578035,0.555556,0.515464,0.051968,0.6,0.2,NaN,NaN,191.038,10.786520,0.056463,150.936,76.216050,0.504956,165.144,170.732,187.78,144.84,0.0,0.0,0.0,0.0,0.139948,0.052938,0.141018,4.48,0.896,1.238839,0.000000,0.0,0.000000,-0.65,-0.130,-5.615385,0.456211,0.5,0.043789
17295,2024-04-06,Espanha Acb,12:00,Zaragoza,Obradoiro CAB,1.50,2.61,167.5,1.87,1.92,-4.5,1.89,4.20,hzJh60At,0.666667,0.383142,0.534759,0.520833,0.049808,0.6,0.4,NaN,NaN,190.806,69.460931,0.364040,268.758,84.576591,0.314694,202.296,333.974,222.00,357.87,0.0,0.0,0.0,0.0,0.381941,0.018657,0.536426,2.02,0.404,1.237624,0.000000,0.0,0.000000,-2.83,-0.566,-2.844523,0.000000,0.0,0.000000
17296,2024-04-06,Eua Nba,16:30,Los Angeles Lakers,Cleveland Cavaliers,1.54,2.26,224.5,1.80,2.01,-4.5,1.93,3.27,E7249e33,0.649351,0.442478,0.555556,0.497512,0.091829,0.6,0.2,NaN,NaN,227.732,47.504106,0.208597,180.374,30.155282,0.167182,228.566,161.908,253.50,180.79,0.0,0.0,0.0,0.0,0.267956,0.077949,0.364432,2.34,0.468,1.153846,0.599498,0.7,0.100502,-2.52,-0.504,-2.500000,0.589800,0.6,0.010200
17297,2024-04-06,Austrália Nbl1 Leste,04:30,Sydney Comets,Albury-Wodonga Bandits,1.78,1.89,164.5,1.80,1.94,-3.5,1.98,2.03,vBZyVItM,0.561798,0.529101,0.555556,0.515464,0.090898,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.042388,0.052938,0.017634,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17298,2024-04-06,Austrália Nbl1 Leste,03:30,Inner West Bulls F,Hills Hornets F,2.93,1.37,134.5,1.85,1.85,-4.5,2.05,1.81,dGwAqZIk,0.341297,0.729927,0.540541,0.540541,0.071224,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.513064,0.000000,0.087930,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17299,2024-04-06,Austrália Nbl1 Sul,05:30,Casey Cavaliers,Dandenong,2.31,1.57,170.5,1.85,1.85,-4.5,1.94,2.01,SOsiVeBo,0.432900,0.636943,0.540541,0.540541,0.069843,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,147.84,0.0,0.0,0.0,0.0,0.269721,0.000000,0.025062,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17300,2024-04-06,Austrália Nbl1 Sul,03:30,Casey Cavaliers F,Dandenong Rangers F,1.57,2.31,155.5,1.86,1.84,-5.5,1.88,2.08,bRU16O6o,0.636943,0.432900,0.537634,0.543478,0.069843,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.269721,0.007644,0.071425,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17301,2024-04-06,Austrália Nbl1 Central,05:15,Norwood Flames,Sturt Sabres,2.19,1.63,174.5,1.87,1.79,1.5,2.04,1.49,xnD4U9ZB,0.456621,0.613497,0.534759,0.558659,0.070118,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,162.84,0.00,0.0,0.0,0.0,0.0,0.207319,0.030912,0.220345,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
17302,2024-04-06,Austrália Nbl1 Oeste,08:00,East Perth Eagles,Warwick Senators,1.94,1.78,179.5,1.80,1.90,1

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Brasil Nbb,Fortaleza B.C.,R10 Score Vasco da Gama,153.5,1.85,1.0000,Over
1,12:00,Espanha Acb,Zaragoza,Obradoiro CAB,167.5,1.87,1.0000,Over
2,16:30,Eua Nba,Los Angeles Lakers,Cleveland Cavaliers,224.5,1.80,1.0000,Over
3,04:30,Austrália Nbl1 Leste,Sydney Comets,Albury-Wodonga Bandits,164.5,1.80,0.9976,Over
4,03:30,Austrália Nbl1 Leste,Inner West Bulls F,Hills Hornets F,134.5,1.85,0.7647,Over
5,05:30,Austrália Nbl1 Sul,Casey Cavaliers,Dandenong,170.5,1.85,0.9995,Over
6,03:30,Austrália Nbl1 Sul,Casey Cavaliers F,Dandenong Rangers F,155.5,1.86,0.9997,Over
7,12:00,Áustria Liga Zweite,Worthersee Piraten,Wagram,143.5,1.85,1.0000,Over
8,13:30,Espanha Leb Ouro,Fuenlabrada,Alicante,156.5,1.83,0.9970,Over
9,15:00,Espanha Leb Plata,L'Hospitalet,Mataro,149.5,1.83,0.9738,Over
